In [1]:
import PyQt6.QtCore
import os
os.environ["QT_API"] = "pyqt6"
import matplotlib.pyplot as plt

# Use qt backend for matplotlab to use interactive mne plots
%matplotlib qt

import mne 
import analysis.processing
import pandas as pd
import csv 
import os
from config import Config
configObj = Config()
from mne_connectivity import spectral_connectivity_time
import numpy as np
configss = configObj.getConfigSnapshot()
from tqdm import tqdm
import tools.helpers
from scipy import stats

mne.set_log_level(verbose='WARNING', return_old_level=False, add_frames=None)

In [7]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 200 # 200 

In [20]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, wilcoxon
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

# List of participant numbers excluding specified ones
participant_numbers = [el for el in range(1, 32) if el not in [5, 13, 14, 16, 17, 20, 31]]

# Initialize an empty list to store DataFrames
dataframes = []

for pnum in tqdm(participant_numbers, desc="Loading Data"):
    participant_data_path = f"{pnum}.csv"
    path_qa = os.path.join(configss['root'], configss['data_completion'], participant_data_path)
    
    # Read and append the DataFrame
    df_participant = pd.read_csv(path_qa)
    dataframes.append(df_participant)

# Concatenate all DataFrames into one combined DataFrame
combined_df = pd.concat(dataframes, axis=0).reset_index(drop=True)

# Function to replace outliers with median
def replace_outliers(df, group_col, value_col):
    df_copy = df.copy()
    medians = df.groupby(group_col)[value_col].transform('median')
    
    Q1 = df[value_col].quantile(0.25)
    Q3 = df[value_col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Replace outliers with median
    df_copy.loc[(df[value_col] < lower_bound) | (df[value_col] > upper_bound), value_col] = medians
    return df_copy

# Replace outliers in the combined data
cleaned_df = replace_outliers(combined_df, 'BlockType', 'completion')

# Aggregate the data
aggregated_df = cleaned_df.groupby(['PID', 'BlockType'])['completion'].mean().reset_index()

# Verify no duplicates
duplicate_counts = aggregated_df.duplicated(subset=['PID', 'BlockType']).sum()
print(f"Number of duplicate PID-BlockType combinations after aggregation: {duplicate_counts}")

if duplicate_counts > 0:
    raise ValueError("Duplicates found after aggregation. Please check your data.")

# Shapiro-Wilk normality test on the aggregated data
shapiro_test = aggregated_df.groupby('BlockType')['completion'].apply(lambda x: shapiro(x)[1])
print(f"\nShapiro-Wilk p-values per BlockType:\n{shapiro_test}")

# Check normality
normality = shapiro_test.all()  # True if all p-values > 0.05
print(f"\nNormality across all BlockTypes: {normality}\n")

if normality:
    # Perform Repeated Measures ANOVA using pingouin
    aov = pg.rm_anova(
        data=aggregated_df,
        dv='completion',
        within='BlockType',
        subject='PID',
        detailed=True
    )
    print("ANOVA Results:")
    print(aov)
    
    # Extract partial eta squared
    partial_eta_squared = aov.loc[aov['Source'] == 'BlockType', 'ng2'].values[0]
    print(f"\nPartial Eta Squared: {partial_eta_squared:.4f}")
    
    effect_size = partial_eta_squared
else:
    # Wilcoxon signed-rank test (assuming two BlockTypes)
    block_types = aggregated_df['BlockType'].unique()
    if len(block_types) != 2:
        raise ValueError("Wilcoxon test requires exactly two BlockType conditions.")
    
    group1, group2 = block_types
    completion1 = aggregated_df[aggregated_df['BlockType'] == group1]['completion']
    completion2 = aggregated_df[aggregated_df['BlockType'] == group2]['completion']
    
    wilcoxon_test_stat, wilcoxon_pvalue = wilcoxon(completion1, completion2)
    
    # Compute Z-statistic using normal approximation (due to ties)
    N = len(completion1)
    mean_rank = (N * (N + 1)) / 4
    std_dev = np.sqrt((N * (N + 1) * (2 * N + 1)) / 24)
    z_stat = (wilcoxon_test_stat - mean_rank) / std_dev
    
    # Effect size (r = Z / sqrt(N))
    effect_size_corrected = abs(z_stat / np.sqrt(N))
    
    print(f"\nWilcoxon Signed-Rank Test Statistic: {wilcoxon_test_stat}")
    print(f"Wilcoxon P-Value: {wilcoxon_pvalue:.4f}")
    print(f"Effect Size (r): {effect_size_corrected:.4f}")
    
    effect_size = effect_size_corrected

# Plot the average completion for each BlockType with 95% CI
plt.figure(figsize=(10, 6))
sns.barplot(x='BlockType', y='completion', data=aggregated_df, ci=95, palette='viridis')
plt.title('Average Completion by Block Type')
plt.xlabel('Block Type')
plt.ylabel('Average Completion')
plt.tight_layout()
plt.show()

# Calculate descriptive statistics
mean_completion = aggregated_df.groupby('BlockType')['completion'].mean()
std_completion = aggregated_df.groupby('BlockType')['completion'].std()

# Display the requested data
if normality:
    display_data = {
        'Mean Completion': mean_completion,
        'Std Deviation': std_completion,
        'Partial Eta Squared': [effect_size] * len(mean_completion)
    }
else:
    display_data = {
        'Mean Completion': mean_completion,
        'Std Deviation': std_completion,
        'Wilcoxon Test Statistic': [wilcoxon_test_stat],
        'Wilcoxon P-Value': [wilcoxon_pvalue],
        'Effect Size (r)': [effect_size_corrected]
    }

display_df = pd.DataFrame(display_data)
print("\nDescriptive Statistics and Effect Size:")
print(display_df)


Loading Data: 100%|██████████| 24/24 [00:00<00:00, 1135.45it/s]

Number of duplicate PID-BlockType combinations after aggregation: 0

Shapiro-Wilk p-values per BlockType:
BlockType
D     0.190438
ND    0.000067
Name: completion, dtype: float64

Normality across all BlockTypes: True

ANOVA Results:
      Source          SS  DF          MS          F     p-unc       ng2  eps
0  BlockType  255.616679   1  255.616679  14.688782  0.000852  0.139885  1.0
1      Error  400.249902  23   17.402170        NaN       NaN       NaN  NaN

Partial Eta Squared: 0.1399

Descriptive Statistics and Effect Size:
           Mean Completion  Std Deviation  Partial Eta Squared
BlockType                                                     
D                91.542969       6.417297             0.139885
ND               96.158312       5.210947             0.139885



C:\Users\erangad\AppData\Local\Temp\ipykernel_24528\2225967126.py:109: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=('ci', 95)` for the same effect.

  sns.barplot(x='BlockType', y='completion', data=aggregated_df, ci=95, palette='viridis')
C:\Users\erangad\AppData\Local\Temp\ipykernel_24528\2225967126.py:109: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='BlockType', y='completion', data=aggregated_df, ci=95, palette='viridis')


In [21]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, wilcoxon, ttest_rel
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
from statsmodels.stats.multitest import multipletests

# -----------------------------
# 1. Data Loading and Cleaning
# -----------------------------

# List of participant numbers excluding specified ones
participant_numbers = [el for el in range(1, 32) if el not in [5, 13, 14, 16, 17, 20, 31]]

# Initialize an empty list to store DataFrames
dataframes = []

for pnum in tqdm(participant_numbers, desc="Loading Data"):
    participant_data_path = f"{pnum}.csv"
    path_qa = os.path.join(configss['root'], configss['data_completion'], participant_data_path)
    
    # Check if file exists
    if not os.path.exists(path_qa):
        print(f"Warning: File {path_qa} does not exist. Skipping participant {pnum}.")
        continue
    
    # Read and append the DataFrame
    try:
        df_participant = pd.read_csv(path_qa)
        df_participant['PID'] = pnum  # Ensure 'PID' column exists
        dataframes.append(df_participant)
    except Exception as e:
        print(f"Error reading {path_qa}: {e}")

# Check if any data was loaded
if not dataframes:
    raise ValueError("No participant data loaded. Please check file paths and data.")

# Concatenate all DataFrames into one combined DataFrame
combined_df = pd.concat(dataframes, axis=0).reset_index(drop=True)

# -----------------------------
# 2. Outlier Replacement
# -----------------------------

def replace_outliers(df, group_col, value_col):
    """
    Replace outliers in 'value_col' within each 'group_col' group with the group's median.
    Outliers are defined as values below Q1 - 1.5*IQR or above Q3 + 1.5*IQR.
    """
    df_copy = df.copy()
    medians = df.groupby(group_col)[value_col].transform('median')
    
    Q1 = df[value_col].quantile(0.25)
    Q3 = df[value_col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Replace outliers with median
    outliers = (df[value_col] < lower_bound) | (df[value_col] > upper_bound)
    num_outliers = outliers.sum()
    print(f"Replacing {num_outliers} outliers in '{value_col}' within each '{group_col}' group.")
    df_copy.loc[outliers, value_col] = medians[outliers]
    
    return df_copy

# Replace outliers in the combined data
cleaned_df = replace_outliers(combined_df, 'BlockType', 'completion')

# -----------------------------
# 3. Data Aggregation
# -----------------------------

# Aggregate the data by computing mean 'completion' per PID and BlockType
aggregated_df = cleaned_df.groupby(['PID', 'BlockType'])['completion'].mean().reset_index()

# Verify no duplicates
duplicate_counts = aggregated_df.duplicated(subset=['PID', 'BlockType']).sum()
print(f"\nNumber of duplicate PID-BlockType combinations after aggregation: {duplicate_counts}")

if duplicate_counts > 0:
    raise ValueError("Duplicates found after aggregation. Please check your data.")

# Ensure that each PID has data for all BlockTypes
block_types = aggregated_df['BlockType'].unique()
block_type_counts = aggregated_df.groupby('PID')['BlockType'].nunique()

missing_blocktypes = block_type_counts[block_type_counts != len(block_types)].index.tolist()
if missing_blocktypes:
    print(f"\nParticipants missing some BlockTypes: {missing_blocktypes}")
    # Exclude these participants from the analysis
    aggregated_df = aggregated_df[~aggregated_df['PID'].isin(missing_blocktypes)]
    print(f"Excluded participants missing BlockTypes. Remaining participants: {aggregated_df['PID'].nunique()}")

# -----------------------------
# 4. Normality Testing on Differences
# -----------------------------

# Pivot the data to have one row per participant with columns for each BlockType
pivot_df = aggregated_df.pivot(index='PID', columns='BlockType', values='completion')

# Check the number of BlockTypes
num_blocktypes = len(block_types)
print(f"\nNumber of BlockTypes: {num_blocktypes}")

if num_blocktypes == 2:
    # Exactly two BlockTypes: proceed with paired tests
    block1, block2 = block_types
    print(f"Block Types identified for paired comparison: '{block1}' and '{block2}'")
    
    # Calculate differences
    pivot_df['Difference'] = pivot_df[block1] - pivot_df[block2]
    
    # Shapiro-Wilk test on differences
    shapiro_stat, shapiro_p = shapiro(pivot_df['Difference'])
    print(f"\nShapiro-Wilk Test for Differences between '{block1}' and '{block2}':")
    print(f"Statistic = {shapiro_stat:.4f}, p-value = {shapiro_p:.4f}")
    
    # Check normality based on differences
    normality = shapiro_p > 0.05
    print(f"\nNormality of Differences: {'Pass' if normality else 'Fail'}")
else:
    # More than two BlockTypes: consider Repeated Measures ANOVA or Friedman Test
    print("\nMore than two BlockTypes detected. Repeated Measures ANOVA will be performed assuming normality.")
    # Proceed with Repeated Measures ANOVA without normality assumption
    # Alternatively, implement a Friedman Test for non-parametric analysis
    # For simplicity, we'll proceed with ANOVA but note the assumption
    differences = pivot_df[block_types].diff(axis=1).iloc[:, -1].dropna()
    shapiro_stat, shapiro_p = shapiro(differences)
    print(f"\nShapiro-Wilk Test for Differences among BlockTypes: Statistic = {shapiro_stat:.4f}, p-value = {shapiro_p:.4f}")
    
    normality = shapiro_p > 0.05
    print(f"\nNormality of Differences: {'Pass' if normality else 'Fail'}")

# -----------------------------
# 5. Statistical Testing
# -----------------------------

if num_blocktypes == 2:
    if normality:
        print("\nData is normally distributed. Performing Paired t-test.")
        # Perform Paired t-test
        t_stat, t_p_value = ttest_rel(pivot_df[block1], pivot_df[block2])
        print(f"\nPaired t-test Results:")
        print(f"t-statistic = {t_stat:.4f}, p-value = {t_p_value:.4f}")
        
        # Calculate Cohen's d for paired samples
        differences = pivot_df['Difference']
        cohen_d = differences.mean() / differences.std(ddof=1)
        print(f"Cohen's d = {cohen_d:.4f}")
        
        # Effect size as per t-test
        effect_size = cohen_d
    else:
        print("\nData is not normally distributed. Performing Wilcoxon signed-rank test.")
        # Perform Wilcoxon signed-rank test
        # Remove zero differences as Wilcoxon cannot handle them
        non_zero = pivot_df['Difference'] != 0
        completion1 = pivot_df.loc[non_zero, block1]
        completion2 = pivot_df.loc[non_zero, block2]
        
        wilcoxon_test_stat, wilcoxon_pvalue = wilcoxon(completion1, completion2, alternative='two-sided')
        print(f"\nWilcoxon Signed-Rank Test Results:")
        print(f"Statistic = {wilcoxon_test_stat}, p-value = {wilcoxon_pvalue:.4f}")
        
        # Calculate Effect Size (Rank Biserial Correlation) using pingouin
        # Since Scipy does not provide Z-statistic directly, we use pingouin for effect size
        effect_size = pg.compute_effsize(completion1, completion2, paired=True, eftype='r')
        print(f"Effect Size (Rank Biserial Correlation) = {effect_size:.4f}")
else:
    if normality:
        print("\nData is normally distributed. Performing Repeated Measures ANOVA.")
        # Perform Repeated Measures ANOVA using pingouin
        aov = pg.rm_anova(
            data=aggregated_df,
            dv='completion',
            within='BlockType',
            subject='PID',
            detailed=True
        )
        print("\nANOVA Results:")
        print(aov)
        
        # Extract partial eta squared
        partial_eta_squared = aov.loc[aov['Source'] == 'BlockType', 'np2'].values[0]
        print(f"\nPartial Eta Squared: {partial_eta_squared:.4f}")
        
        effect_size = partial_eta_squared
    else:
        print("\nData is not normally distributed. Performing Friedman test.")
        from scipy.stats import friedmanchisquare
        
        # Prepare data for Friedman test
        friedman_data = [pivot_df[bt].dropna() for bt in block_types]
        
        # Perform Friedman test
        friedman_stat, friedman_pvalue = friedmanchisquare(*friedman_data)
        print(f"\nFriedman Test Results:")
        print(f"Statistic = {friedman_stat:.4f}, p-value = {friedman_pvalue:.4f}")
        
        # Calculate Effect Size (Partial Eta Squared)
        # Formula: η² = (chi²) / (N * (k - 1))
        N = pivot_df.shape[0]
        k = num_blocktypes
        partial_eta_squared = friedman_stat / (N * (k - 1))
        print(f"Partial Eta Squared = {partial_eta_squared:.4f}")
        
        effect_size = partial_eta_squared

# -----------------------------
# 6. Visualization
# -----------------------------

# Plot the average completion for each BlockType with 95% CI
plt.figure(figsize=(10, 6))
sns.barplot(x='BlockType', y='completion', data=aggregated_df, ci=95, palette='viridis')
plt.title('Average Completion by Block Type')
plt.xlabel('Block Type')
plt.ylabel('Average Completion')
plt.tight_layout()
plt.show()

# -----------------------------
# 7. Descriptive Statistics and Effect Size Reporting
# -----------------------------

# Calculate descriptive statistics
mean_completion = aggregated_df.groupby('BlockType')['completion'].mean()
std_completion = aggregated_df.groupby('BlockType')['completion'].std()

# Display the requested data
if num_blocktypes == 2:
    if normality:
        display_data = {
            'Mean Completion': mean_completion,
            'Std Deviation': std_completion,
            'Effect Size (Cohen\'s d)': [effect_size] * len(mean_completion)
        }
    else:
        display_data = {
            'Mean Completion': mean_completion,
            'Std Deviation': std_completion,
            'Wilcoxon Test Statistic': [wilcoxon_test_stat],
            'Wilcoxon P-Value': [wilcoxon_pvalue],
            'Effect Size (r)': [effect_size]
        }
else:
    if normality:
        display_data = {
            'Mean Completion': mean_completion,
            'Std Deviation': std_completion,
            'Partial Eta Squared': [effect_size] * len(mean_completion)
        }
    else:
        display_data = {
            'Mean Completion': mean_completion,
            'Std Deviation': std_completion,
            'Friedman Test Statistic': [friedman_stat],
            'Friedman P-Value': [friedman_pvalue],
            'Effect Size (Partial Eta Squared)': [effect_size]
        }

display_df = pd.DataFrame(display_data)
print("\nDescriptive Statistics and Effect Size:")
print(display_df)


Loading Data: 100%|██████████| 24/24 [00:00<00:00, 904.46it/s]
C:\Users\erangad\AppData\Local\Temp\ipykernel_24528\2087868017.py:220: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=('ci', 95)` for the same effect.

  sns.barplot(x='BlockType', y='completion', data=aggregated_df, ci=95, palette='viridis')
C:\Users\erangad\AppData\Local\Temp\ipykernel_24528\2087868017.py:220: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='BlockType', y='completion', data=aggregated_df, ci=95, palette='viridis')


Replacing 16 outliers in 'completion' within each 'BlockType' group.

Number of duplicate PID-BlockType combinations after aggregation: 0

Number of BlockTypes: 2
Block Types identified for paired comparison: 'D' and 'ND'

Shapiro-Wilk Test for Differences between 'D' and 'ND':
Statistic = 0.9387, p-value = 0.1526

Normality of Differences: Pass

Data is normally distributed. Performing Paired t-test.

Paired t-test Results:
t-statistic = -3.8326, p-value = 0.0009
Cohen's d = -0.7823

Descriptive Statistics and Effect Size:
           Mean Completion  Std Deviation  Effect Size (Cohen's d)
BlockType                                                         
D                91.542969       6.417297                -0.782325
ND               96.158312       5.210947                -0.782325


In [19]:
aggregated_df

,PID,BlockType,completion
0,1,D,95.000000
1,1,ND,96.614583
2,2,D,88.307292
3,2,ND,98.151042
4,3,D,93.411458
5,3,ND,100.000000
6,4,D,90.833333
7,4,ND,99.401042
8,6,D,91.458333
9,6,ND,90.325521
